<a href="https://colab.research.google.com/github/JA-Bar/nlp-depression/blob/master/Prediction_suicidal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!gdown https://drive.google.com/uc?id=1FaDZvPevPZaw2IIUeNx9OwEmjL6Fe-Fe -O nlp.zip

In [ ]:
import zipfile
#!unzip '/content/nlp.zip'

In [ ]:
#!pip install -q tensorflow-text
#!pip install -q tf-models-official
#!pip install -q sklearn
#!pip install tensorflow-text

In [8]:
import tensorflow as tf
import tensorflow_text as text
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM,SimpleRNN,GRU,RNN
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from numpy import array
from numpy import asarray
from numpy import zeros
import tensorflow_hub as hub
import tensorflow as tf
from official.nlp import optimization 
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import tensorflow as tf
from sklearn.model_selection import train_test_split
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
import pickle

In [9]:
def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)
    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)
    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    return sentence

In [ ]:
def convert2embeding (word):
  with open('/content/content/gdrive/MyDrive/Clasificador/nlp/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
  X = []
  sentences = list(word)
  #print(sentences)
  for sen in sentences:
    X.append(preprocess_text(sen))
  #print(X)
  token_word=tokenizer.texts_to_sequences(X[0])
  #print(token_word)
  final=[]
  for tok in token_word:
    try:
      final.append(tok[0])
    except:
      final.append(0)
  #print(final)
  maxlen = 32
  final_word = pad_sequences([final], padding='post', maxlen=maxlen)
  #print(final_word)
  return final_word
word2='Beatiful world baby'
x=convert2embeding ([word2])
print(x)

In [15]:
glove_dataset_dep='/content/content/gdrive/MyDrive/Clasificador/nlp/glove_suicidal.h5'
fasttext_dataset_dep='/content/content/gdrive/MyDrive/Clasificador/nlp/fasttext_dataset_hugging.h5'

bert_sucidal = tf.saved_model.load('/content/content/gdrive/MyDrive/Clasificador/nlp/bert_suicidal')
bert_sadness = tf.saved_model.load('/content/content/gdrive/MyDrive/Clasificador/nlp/bert_sadness')

glove = keras.models.load_model(glove_dataset_dep)
fasttext = keras.models.load_model(fasttext_dataset_dep)

In [ ]:
def predictions (word2pred,glove_model,fasttext_model,bert_sucidal,bert_sadness):
  final_pred=[]
  bert_result=tf.sigmoid(bert_sucidal(tf.constant([word2pred])))
  bert_result=bert_result.numpy()[0][0]
  #print(bert_result)

  bert_sad_result=tf.sigmoid(bert_sadness(tf.constant([word2pred])))
  bert_sad_result=bert_sad_result.numpy()[0][0]
  #print(bert_sad_result)

  #print(bert_result >= .60)

  if bert_result > .60:
    #print('bert_sucidal')
    final_pred.append([bert_result,1-bert_result])
    final_pred=final_pred[0]
  else:
    #print('1')
    convertion=convert2embeding([word2pred])
    #print('2')
    glove_pred = glove_model.predict(convertion)
    #print('3')
    fastext_pred = fasttext_model.predict(convertion)
    #print('4')
    for pred in range(len(glove_pred)):
      #print('pred=', pred)
      final_pred.append((glove_pred[pred]+fastext_pred[pred])/2)
      #print('final_pred=', final_pred)
      #print("final_pred = ",final_pred)
      #print("glove_pred[pred] = ",glove_pred[pred])
      #print("fastext_pred[pred] = ",fastext_pred[pred])
    final_pred[0][1]=(final_pred[0][1]+bert_sad_result)/2
    final_pred[0][0]=(final_pred[0][0])/1.1
    final_pred=final_pred[0]
  return final_pred

In [ ]:
frase='i want to '
x=predictions (frase,glove,fasttext,bert_sucidal,bert_sadness)
print(x)